In [3]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import warnings
import spacy
from optimized_anchor import anchor_text, anchor_base
import pickle
import myUtils
from myUtils import *
from models.utils import *
from models.dataset_loader import *
import datetime
import time
import argparse
import os
import sys
sys.path.append('models')
# when apply torchscript to models sometimes
#torch._C._jit_set_texpr_fuser_enabled(False)

warnings.simplefilter("ignore")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_name = 'huawei-noah/TinyBERT_General_4L_312D'
%load_ext line_profiler

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
# can be sentiment/spam/dilemma
dataset_name = 'corona'
ds = get_ds(dataset_name)

In [7]:
anchor_examples, _  = preprocess_examples(ds, 200)
f_name = f'results/mp/deberta/corona/confidence/42/desired-0.1' 
anchor_examples = pickle.load(open(f"{f_name}/anchor_examples.pickle", "rb" ))

Filter:   0%|          | 0/12542 [00:00<?, ? examples/s]

In [5]:
nlp = spacy.load('en_core_web_sm')
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast = False)
myUtils.tokenizer = tokenizer

In [9]:
ignored = get_ignored(anchor_examples)
normal_occurences = get_occurences(anchor_examples)
bg = BestGroup('check', normal_occurences, filter_anchors = False, desired_optimize = False)

## notice!

In [10]:
ignored = []

In [13]:
anchor_examples[5348]

'everyone handleing the covid 19 outbreak: â\x95 stock up on food â\x95 stook up on supplies â\x95 panic!! â\x95 buying 25 packets of toilet rolls then there is my best mate  : â\x91ninja training day 3â\x92'

In [11]:
examples = anchor_examples[5348:5349]

In [12]:
!nvidia-smi

Mon Jun 19 16:41:56 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.54       Driver Version: 510.54       CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Quadro RTX 6000     Off  | 00000000:AF:00.0 Off |                  Off |
| 33%   44C    P2    66W / 260W |   2207MiB / 24576MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [13]:
optimize = True
anchor_text.AnchorText.set_optimize(optimize)
explainer = anchor_text.AnchorText(nlp, ['positive', 'negative'], use_unk_distribution=False)

In [ ]:
tokenized = explainer.tg.bert_tokenizer(["i [MASK] to [MASK] party yesterday"], 
          add_special_tokens=True, return_tensors="pt", padding=True)["input_ids"]

In [ ]:
tokenized_seperate = [explainer.tg.bert_tokenizer.encode("i went to a party yesterday", 
          add_special_tokens=True) for _ in range(100)]
tokenized_seperate = [torch.tensor([t], device=device) for t in tokenized_seperate]

In [14]:
model = load_model(f'models/deberta/{dataset_name}/model').to(device).eval()
tokenizer = AutoTokenizer.from_pretrained('microsoft/deberta-v3-small', 
                                          use_fast = False)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [19]:
torch._C._jit_set_texpr_fuser_enabled(False)
my_utils = TextUtils(examples, explainer, predict_sentences, ignored, optimize=optimize)
myUtils.model = model
myUtils.tokenizer = tokenizer
anchor_base.AnchorBaseBeam.best_group = bg
set_seed()
# %prun -s cumtime -T check/profile.txt my_utils.compute_explanations(list(range(len(examples))))
exps = my_utils.compute_explanations(list(range(len(examples))))

number 0
{'feature': [35], 'mean': [1.0], 'precision': [1.0], 'coverage': [0.0], 'examples': [], 'all_precision': 0, 'num_preds': 100001}
['best']
[1.0]


In [26]:
[e.names[0] for e in exps]

['everyone',
 'handle',
 'the',
 'co',
 ':',
 '[UNK]',
 'stock',
 'up',
 'on',
 'food',
 '[UNK]',
 'on',
 'supplies',
 '[UNK]',
 '!',
 '!',
 '25',
 'packets',
 'of',
 'rolls',
 'then',
 'there',
 'is',
 'my',
 'best',
 'mate',
 ':',
 '[UNK]',
 'training',
 'day',
 '[UNK]']

In [154]:
torch._C._jit_set_texpr_fuser_enabled(False)
my_utils = TextUtils(examples, explainer, predict_sentences, ignored, optimize=optimize)
myUtils.model = model
myUtils.tokenizer = tokenizer
anchor_base.AnchorBaseBeam.best_group = bg
set_seed()
%prun -s cumtime -T check/profile.txt my_utils.compute_explanations(list(range(len(examples))))

number 0
number 1
[0.9807692307692307]
[0.9669421487603306]
number 2
[0.971830985915493]
number 3
number 4
number 5
[0.9702970297029703]
number 6
number 7
number 8
number 9
[0.9669421487603306]
[0.971830985915493]
number 10
[1.0]
[0.9590643274853801]
[0.9803921568627451]
number 11
number 12
[0.967741935483871]
number 13
[0.9645390070921985]
number 14
[1.0]
[0.9803921568627451]
number 15
number 16
number 17
[1.0]
number 18
[1.0]
[1.0]
number 19
number 20
[1.0]
number 21
[1.0]
number 22
number 23
number 24
number 25
[1.0]
number 26
[1.0]
[1.0]
number 27
number 28
 
*** Profile printout saved to text file 'check/profile.txt'.


         63160161 function calls (62897111 primitive calls) in 65.819 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      3/1    0.000    0.000   65.852   65.852 {built-in method builtins.exec}
        1    0.000    0.000   65.852   65.852 <string>:1(<module>)
        1    0.000    0.000   65.852   65.852 myUtils.py:274(compute_explanations)
       29    1.858    0.064   65.849    2.271 myUtils.py:271(get_exp)
       29    0.002    0.000   63.992    2.207 anchor_text.py:269(explain_instance)
       29    0.021    0.001   61.053    2.105 anchor_base.py:255(anchor_beam)
     2489    0.264    0.000   60.807    0.024 anchor_text.py:225(sample_fn)
     2409    0.003    0.000   60.037    0.025 anchor_base.py:217(<lambda>)
     2409    0.079    0.000   60.034    0.025 anchor_base.py:186(complete_sample_fn)
     2489    0.380    0.000   37.103    0.015 anchor_text.py:118(sample)
     3395    0.066    0.000   26.299    0.008 anc

In [ ]:
###### my_utils = TextUtils(anchor_examples, test, explainer, predict_sentences, ignored,f"profile.pickle")
#%lprun -s -m modified_anchor.anchor_text -m modified_anchor.anchor_base -m myUtils -T profile.txt  my_utils.compute_explanations(list(range(len(anchor_examples))))

In [ ]:
print(datetime.datetime.now())